[View in Colaboratory](https://colab.research.google.com/github/dkatsios/semantic_segmentation/blob/master/voc2012_helpers.ipynb)

In [5]:
import numpy as np
import cv2
from time import time
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [0]:
def get_lists_from_folders(train_list_path, val_list_path, imgs_folder, classes_folder):
  train_list = []
  with open(train_list_path) as f:
    for line in f:
      line = line.strip()
      if len(line) > 1:
        train_list.append(line)
        
  train_imgs_list = [imgs_folder + name + '.jpg' for name in train_list]
  train_classes_list = [classes_folder + name + '.png' for name in train_list]
  
  val_list = []
  with open(val_list_path) as f:
    for line in f:
      line = line.strip()
      if len(line) > 1:
        val_list.append(line)
        
  val_imgs_list = [imgs_folder + name + '.jpg' for name in val_list]
  val_classes_list = [classes_folder + name + '.png' for name in val_list]
  
  train_lists = train_imgs_list, train_classes_list
  val_lists = val_imgs_list, val_classes_list
  
  return train_lists, val_lists

In [0]:
def imgs_generator(rgb_imgs, num_classes, batch_size, out_resized_levels, segmentation_classes):
  while True:
    inds = np.random.randint(0, rgb_imgs.shape[0], batch_size)
    batch_imgs = rgb_imgs[inds]
    batch_classes = get_resized(num_classes[inds], out_resized_levels, segmentation_classes)[::-1]
    yield batch_imgs, batch_classes

In [0]:
def get_resized(num_classes, out_resized_levels, segmentation_classes):
  batch_size = num_classes.shape[0]
  or_size = num_classes.shape[1:]
  assert len(or_size) == 2, num_classes.shape
  if out_resized_levels == 0:
    return make_one_hot(num_classes, segmentation_classes)
  resized_classes = [make_one_hot(num_classes, segmentation_classes)]
  for i in range(out_resized_levels):
    size = or_size[0] // (2 ** (i+1)), or_size[1] // (2 ** (i+1))
    resized = np.zeros((batch_size, *size))
    for j in range(batch_size):
      resized[j] = cv2.resize(num_classes[j], size, interpolation=cv2.INTER_NEAREST)
    resized = make_one_hot(resized, segmentation_classes)
    resized_classes.append(resized)
  return resized_classes

In [0]:
def get_cmap_dict(reversed=False):
  def color_map(N=256, normalized=False):
    
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3
        cmap[i] = np.array([r, g, b])
    cmap = cmap/255 if normalized else cmap
    return cmap
  
  cmap = color_map()
  cmap_dict = dict()
  for i in range(cmap.shape[0]):
    if reversed:
      cmap_dict[i] = tuple(cmap[i].astype(np.int64))
    else:
      cmap_dict[tuple(cmap[i].astype(np.int64))] = i
  return cmap_dict

In [0]:
def get_imgs_classes_arrays(imgs_list, classes_list, img_shape):
  assert len(imgs_list) == len(classes_list)

  def color_to_nums(num_classes, cmap_dict):
    nums = np.zeros(num_classes.shape[:-1], dtype=np.int64)
    mul = np.array([1, 10, 100])
    factor = np.tile(mul, (*num_classes.shape[:-1], 1))
    nums = np.multiply(num_classes, factor)
    nums = np.sum(nums, axis=-1)
    for key, value in cmap_dict.items():
      key = np.asarray(key).astype(np.int64)
      key = np.sum(np.multiply(key, mul))
      nums[nums == key] = value
    return nums
  
  print('start constructing arrays')
  start = time()
  cmap_dict = get_cmap_dict()
  
  img_shape = img_shape
  rgb_imgs = np.zeros((len(imgs_list), *img_shape), dtype=np.uint8)
  num_classes = np.zeros((len(classes_list), *img_shape[:-1]), dtype=np.uint8)
  
  for ind in range(len(imgs_list)):
    img = cv2.imread(imgs_list[ind])
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    rgb_resized = cv2.resize(rgb_img, img_shape[:-1])
    rgb_imgs[ind] = rgb_resized
    
    cls = cv2.imread(classes_list[ind])
    rgb_cls = cv2.cvtColor(cls, cv2.COLOR_BGR2RGB)
    cls_resized = cv2.resize(rgb_cls, img_shape[:-1], interpolation=cv2.INTER_NEAREST)
    num_class = color_to_nums(cls_resized, cmap_dict)
    num_classes[ind] = num_class
  print('arrays constructed. time: %d secs' % int(time() - start))
  return rgb_imgs, num_classes

In [0]:
def make_one_hot(batch_labels, segmentation_classes):
  batch_labels[batch_labels == 255] = segmentation_classes
  categorical_labels = to_categorical(batch_labels, num_classes=segmentation_classes+1)
  return categorical_labels

In [0]:
def get_class_weight(seg_classes, background_ratio = 1/5):
  class_weight = dict()
  class_weight[0] = background_ratio / seg_classes
  for i in range(1, seg_classes):
    class_weight[i] = 1 / seg_classes + (1 - background_ratio) / seg_classes ** 2
  class_weight[seg_classes] = 0
  return class_weight